In [ ]:
import tensorflow as tf
import os
import datetime as dtime
import time

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

class BasicBlock(tf.keras.layers.Layer):
    def __init__(self,filter_num,stride = 1):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(filter_num,(3,3),strides = stride,padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.Activation('relu')
        
        self.conv2 = tf.keras.layers.Conv2D(filter_num,(3,3),strides = 1,padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()
        
        if stride != 1 :
            self.downsample = tf.keras.Sequential([
                tf.keras.layers.Conv2D(filter_num,(1,1),strides=stride)
            ])
        else:
            self.downsample = lambda x:x
            
    def call(self,inputs,training=None):
        out = self.conv1(inputs)
        out= self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        identity = self.downsample(inputs)
        output = tf.keras.layers.add([out,identity])
        output = tf.nn.relu(output)
        return output
    
class ResNet(tf.keras.Model):
    def __init__(self,layer_dims,num_class = 10):
        super().__init__()
        self.stem = tf.keras.Sequential([
            tf.keras.layers.Conv2D(64,(3,3),strides=(1,1)),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(1,1),padding='same')
        ])
        
        self.layer1 = self.build_resblock(64,layer_dims[0])
        self.layer2 = self.build_resblock(128,layer_dims[1],stride=2)
        self.layer3 = self.build_resblock(256,layer_dims[2],stride=2)
        self.layer4 = self.build_resblock(512,layer_dims[3],stride=2)
        
        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc = tf.keras.layers.Dense(num_class)
        
    def call(self,inputs,training = None):
        x = self.stem(inputs)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        
        x = self.fc(x)
        return x
        
    def build_resblock(self,filter_num,blocks,stride=1):
        res_blocks = tf.keras.Sequential([
            BasicBlock(filter_num,stride)
        ])
        for _ in range(1,blocks):
            res_blocks.add(BasicBlock(filter_num,stride=1))
        return res_blocks
    
def resnet18():
    return ResNet([2,2,2,2])

In [ ]:


os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


def preprocess(x,y):
    x = tf.cast(x,dtype=tf.float32)/255
    y = tf.cast(y,dtype=tf.int32)
    return x,y
    
(x,y),(x_test,y_test) = tf.keras.datasets.cifar100.load_data()
y = tf.squeeze(y,axis=1)
y_test = tf.squeeze(y_test,axis=1)

train_db = tf.data.Dataset.from_tensor_slices((x,y))
train_db = train_db.shuffle(1000).map(preprocess).batch(100)

test_db = tf.data.Dataset.from_tensor_slices((x_test,y_test))
test_db = test_db.map(preprocess).shuffle(1000).batch(100)

model = resnet18()
model.build(input_shape = (None,32,32,3))
model.summary()
learn_rate = 1e-4
epoch_num = 30
optimizer = tf.keras.optimizers.Adam(lr = learn_rate)

current_time = dtime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = '../logs/'+current_time
summary_writer = tf.summary.create_file_writer(log_dir)

pre_loss,cur_loss = 0.,0.
start = dtime.datetime.now()
num_count=0
for epoch in range(epoch_num):
    for step,(x,y) in enumerate(train_db):
        num_count+=1
        with tf.GradientTape() as tape:
            logits = model(x)
            y_onehot = tf.one_hot(y,depth=10)
            
            loss = tf.reduce_mean(tf.losses.categorical_crossentropy(y_onehot,logits,from_logits=True))
            pre_loss = cur_loss
            cur_loss = loss
        
        grads = tape.gradient(loss,model.trainable_variables)
        optimizer.apply_gradients(zip(grads,model.trainable_variables))
        
        if step % 100 == 0:
            with summary_writer.as_default():
                tf.summary.scalar('train_loss',float(loss),step=num_count)
            print('epoch:',epoch,'step:',step,'loss:',loss)
    
    optimizer.lr = learn_rate*(epoch_num-epoch)/epoch_num
    if epoch>20 and cur_loss-pre_loss>0.1:
        break;
        
    total_num = 0
    total_correct = 0
    for x,y in test_db:
        logits = model(x)
        prob = tf.nn.softmax(logits,axis=1)
        pred = tf.cast(tf.argmax(prob,axis=1),dtype=tf.int32)
        correct = tf.reduce_sum(tf.cast(tf.equal(pred,y),dtype=tf.int32))
        
        total_num +=x.shape[0]
        total_correct +=int(correct)
    acc = total_correct/total_num
    with summary_writer.as_default():
        tf.summary.scalar('test_acc',float(acc),step = epoch)
    
    if acc > 0.9:
        break;
    print('epoch:',epoch,'acc:',acc)
    
end = dtime.datetime.now()
print(end - start)